In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import heapq
import os
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
data_path = 'data/'                         
data_1 = 'data_learn_match.csv'
data_2 = 'data_test_match.csv'
data_3 = 'data_target_thread.csv'
raw_data = pd.read_csv(os.path.join(data_path, data_1), header=None)
raw_data.drop(raw_data.columns[2:3], axis=1, inplace=True)
raw_data['values'] = 1

In [3]:
raw_data.columns = ['user_uid', 'thread_uid', 'match_date', 'values']

In [5]:
raw_data['values'] = 15552000/(15552000 + 1570136400 - raw_data['match_date'])

In [6]:
input_list_1 = raw_data['user_uid'].unique()

def scale_user_uid(input_uid):
    return np.where(input_list_1 == input_uid)[0][0]

raw_data['user_uid_renumber'] = raw_data['user_uid'].apply(scale_user_uid)

In [10]:
data_mini = raw_data.copy()

In [11]:
n_users = data_mini['user_uid'].unique().shape[0]
n_items = data_mini['thread_uid'].unique().shape[0]

In [15]:
raw_data_2 = pd.read_csv(os.path.join(data_path, data_2), header=None)

In [16]:
raw_data_2.columns = ['user_uid_test', 'thread_uid_test']
data_2 = raw_data_2.copy()

In [17]:
raw_data_3 = pd.read_csv(os.path.join(data_path, data_3), header=None)

In [18]:
raw_data_3.columns = ['thread_uid_test']
data_3 = raw_data_3.copy()
list_test_threads = data_3['thread_uid_test'].unique().tolist()

In [19]:
list_test_user = data_2['user_uid_test'].unique()
list_match_user = data_mini['user_uid'].unique()

In [20]:
list_drop = list(set(list_test_user).difference(list_match_user))

In [23]:
list_test = [x for x in list_test_user if x not in list_drop]

In [26]:
list_test.sort()

[47,
 51,
 52,
 115,
 156,
 171,
 175,
 178,
 181,
 189,
 195,
 210,
 215,
 222,
 321,
 349,
 373,
 395,
 450,
 461,
 467,
 506,
 548,
 579,
 599,
 601,
 707,
 711,
 731,
 749,
 808,
 816,
 828,
 834,
 850,
 865,
 875,
 915,
 918,
 1015,
 1018,
 1019,
 1028,
 1099,
 1145,
 1164,
 1168,
 1170,
 1214,
 1232,
 1268,
 1316,
 1330,
 1338,
 1355,
 1421,
 1476,
 1499,
 1528,
 1538,
 1553,
 1560,
 1645,
 1649,
 1684,
 1727,
 1789,
 1792,
 1808,
 1838,
 1847,
 1870,
 1903,
 1936,
 2009,
 2047,
 2071,
 2089,
 2103,
 2112,
 2148,
 2219,
 2295,
 2315,
 2385,
 2387,
 2420,
 2453,
 2520,
 2523,
 2539,
 2549,
 2552,
 2566,
 2587,
 2589,
 2597,
 2620,
 2645,
 2677,
 2682,
 2739,
 2872,
 3069,
 3081,
 3142,
 3173,
 3174,
 3186,
 3336,
 3394,
 3431,
 3466,
 3468,
 3541,
 3562,
 3567,
 3659,
 3773,
 3784,
 3836,
 3837,
 3876,
 3929,
 3992,
 4069,
 4107,
 4144,
 4145,
 4159,
 4161,
 4211,
 4243,
 4275,
 4333,
 4371,
 4379,
 4443,
 4466,
 4475,
 4479,
 4541,
 4564,
 4600,
 4616,
 4693,
 4706,
 4771,
 5076,

In [27]:
list_user = list_match_user.tolist()

59012

In [28]:
list_of_idx = []
for user in tqdm(list_test):
    idx = list_user.index(user)
    list_of_idx.append(idx)

100%|█████████████████████████████████████| 2713/2713 [00:17<00:00, 158.66it/s]


[46,
 50,
 51,
 114,
 155,
 170,
 174,
 177,
 180,
 188,
 194,
 209,
 214,
 221,
 320,
 348,
 372,
 393,
 448,
 459,
 465,
 503,
 545,
 576,
 596,
 598,
 701,
 705,
 725,
 743,
 801,
 809,
 821,
 827,
 843,
 858,
 868,
 908,
 911,
 1008,
 1011,
 1012,
 1021,
 1090,
 1135,
 1154,
 1158,
 1160,
 1204,
 1222,
 1258,
 1306,
 1320,
 1328,
 1344,
 1409,
 1463,
 1486,
 1515,
 1525,
 1540,
 1547,
 1632,
 1636,
 1671,
 1714,
 1776,
 1779,
 1795,
 1825,
 1834,
 1857,
 1890,
 1923,
 1995,
 2033,
 2057,
 2075,
 2089,
 2098,
 2133,
 2204,
 2279,
 2299,
 2369,
 2371,
 2404,
 2437,
 2503,
 2506,
 2522,
 2532,
 2535,
 2548,
 2569,
 2571,
 2579,
 2602,
 2627,
 2659,
 2664,
 2720,
 2851,
 3048,
 3060,
 3121,
 3152,
 3153,
 3165,
 3314,
 3372,
 3409,
 3444,
 3446,
 3519,
 3540,
 3545,
 3637,
 3751,
 3762,
 3813,
 3814,
 3852,
 3905,
 3968,
 4045,
 4083,
 4120,
 4121,
 4135,
 4137,
 4186,
 4218,
 4250,
 4308,
 4346,
 4354,
 4418,
 4441,
 4450,
 4454,
 4516,
 4539,
 4575,
 4591,
 4668,
 4681,
 4745,
 5049,

In [32]:
sparse_user_item = sparse.csr_matrix((data_mini['values'].astype(float), (data_mini['user_uid_renumber'], data_mini['thread_uid'])))

In [37]:
sim_cos = []
for i in tqdm(list_of_idx):
    similarities = cosine_similarity(sparse_user_item, sparse_user_item[i:i+1]).reshape(sparse_user_item.shape[0],)
    sim_cos.append(similarities)

100%|██████████████████████████████████████| 2713/2713 [06:10<00:00,  7.32it/s]


[array([0.        , 0.03750893, 0.00374351, ..., 0.        , 0.        ,
        0.        ]),
 array([0.05820545, 0.00380563, 0.0372834 , ..., 0.        , 0.        ,
        0.        ]),
 array([0.        , 0.        , 0.00912603, ..., 0.        , 0.03310475,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.0467577 , 0.        , 0.04398836, ..., 0.        , 0.        ,
        0.        ]),
 array([0.        , 0.00232359, 0.01032258, ..., 0.        , 0.02118849,
        0.        ]),
 array([0.        , 0.        , 0.00048583, ..., 0.        , 0.        ,
        0.        ]),
 array([0.        , 0.        , 0.01889708, ..., 0.        , 0.        ,
        0.        ]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.01228097,
        0.        ]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.05243356,
        0.        ]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.04564799,
        0.        ]),
 array([0.,

In [38]:
#Модуль фильтрации по квантилю или по значению сходства. Если раскомментировать,
#то ниже вместо "25" поставить rec_len[user]
#rec_len = []
#for scores in range(len(sim_cos)):
    #quantile = np.quantile(sim_cos[scores], 0.9995)
    #condition = lambda x: x >= quantile                                 
    #filtered_scores = list(filter(condition, sim_cos[scores]))         
    #len_filtered_scores = len(filtered_scores)                         
    #rec_len.append(len_filtered_scores)

In [66]:
similarity_users = []
for user in tqdm(range(len(sim_cos))):
    a = heapq.nlargest(25, range(len(sim_cos[user])), sim_cos[user].take)  #rec_len[user] вместо числа пользователей, 
    sim = []                                                               #если фильтрация есть
    for idx in a:
        if idx != list_of_idx[user]:
            sim_user = list_user[idx]
            sim.append(sim_user)
        else: 
            pass
    similarity_users.append(sim)

100%|██████████████████████████████████████| 2713/2713 [02:43<00:00, 16.62it/s]


[[56917,
  58561,
  52101,
  20130,
  55306,
  7576,
  40953,
  26719,
  222,
  57592,
  55889,
  56690,
  57018,
  50751,
  53377,
  47871,
  56782,
  55565,
  49232,
  30424,
  42759,
  50444,
  21835,
  5102],
 [4856,
  23782,
  9470,
  37977,
  6390,
  33720,
  39445,
  34533,
  1520,
  35351,
  20130,
  25456,
  27542,
  35425,
  41484,
  9202,
  38967,
  678,
  41528,
  461,
  43567,
  23622,
  1099,
  6805],
 [28857,
  43204,
  6408,
  46802,
  39224,
  35728,
  35414,
  1198,
  12256,
  47762,
  32496,
  38831,
  41417,
  36818,
  39550,
  29814,
  51372,
  43967,
  40804,
  2420,
  26729,
  34250,
  26380,
  34439],
 [54191,
  34054,
  37494,
  8512,
  43266,
  23553,
  47756,
  53208,
  5302,
  56182,
  58385,
  20566,
  16288,
  55436,
  28630,
  45205,
  1945,
  50542,
  55233,
  26584,
  50269,
  53061,
  43455,
  51467],
 [26071,
  43509,
  8205,
  41000,
  25151,
  19177,
  59027,
  33544,
  17212,
  27857,
  23131,
  46623,
  42000,
  8095,
  43968,
  4736,
  1326,
  37

In [67]:
similarity_threads = []
for sim in tqdm(similarity_users):
    list_sim = []
    for user in sim:
        sim_threads = data_mini['thread_uid'].loc[data_mini['user_uid'] == user].tolist()
        list_sim_threads = [x for x in sim_threads if x in list_test_threads]
        if len(list_sim_threads) != 0:
            list_sim=list_sim+list_sim_threads
        else: 
            pass
    similarity_threads.append(list(set(list_sim)))

100%|██████████████████████████████████████| 2713/2713 [20:16<00:00,  2.23it/s]


[[103307,
  103578,
  103216,
  103601,
  103604,
  103608,
  103491,
  103494,
  103625,
  103506,
  103259,
  103643,
  103520,
  103394,
  103269,
  103540,
  103542,
  103286,
  103417],
 [103520,
  103233,
  103427,
  103523,
  103625,
  103311,
  103407,
  103314,
  103506,
  103602,
  103542,
  103608,
  103578,
  103391],
 [103425,
  103427,
  103556,
  103560,
  103432,
  103437,
  103315,
  103443,
  103447,
  103575,
  103449,
  103450,
  103325,
  103326,
  103596,
  103598,
  103471,
  103222,
  103226,
  103483,
  103612,
  103613,
  103486,
  103487,
  103488,
  103489,
  103614,
  103616,
  103234,
  103236,
  103241,
  103242,
  103497,
  103500,
  103501,
  103373,
  103633,
  103508,
  103512,
  103641,
  103515,
  103260,
  103265,
  103530,
  103276,
  103278,
  103535,
  103537,
  103281,
  103285,
  103286,
  103543,
  103417,
  103418,
  103548],
 [103554,
  103301,
  103429,
  103304,
  103565,
  103568,
  103442,
  103457,
  103585,
  103467,
  103596,
  10334

In [71]:
data_2['columns'] = data_2.groupby('user_uid_test')['thread_uid_test'].cumcount()

In [72]:
result = data_2.pivot(index='user_uid_test', columns='columns')

In [73]:
result['concat_col']=result.apply(lambda row: row.dropna().tolist(), axis=1)

In [74]:
df_test_threads = result[['concat_col']]

In [76]:
df_test_threads_f = df_test_threads.drop(list_drop, axis = 0)

,concat_col
columns,
user_uid_test,
47,[103578.0]
51,[103474.0]
52,"[103233.0, 103506.0]"
115,[103442.0]
156,"[103613.0, 103625.0]"
...,...
57232,[103634.0]
57242,[103594.0]


In [77]:
df_test_threads_f['sim_threads'] = similarity_threads

In [79]:
raw_data_seen = raw_data.loc[raw_data['user_uid'].isin(list_test)]

,user_uid,thread_uid,match_date,values,user_uid_renumber
9361,47,182,1463076644,0.126839,46
9362,47,551,1558122978,0.564185,46
9363,47,1037,1420920939,0.094388,46
9364,47,4469,1556642889,0.535436,46
9365,47,5985,1565599253,0.774149,46
...,...,...,...,...,...
9801120,57287,101109,1566277450,0.801197,57133
9801121,57287,101791,1566810198,0.823807,57133
9801122,57287,102205,1568115882,0.885018,57133
9801123,57287,102589,1568616892,0.910992,57133


In [80]:
raw_data_seen['columns'] = raw_data_seen.groupby('user_uid')['thread_uid'].cumcount()

C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
result_2 = raw_data_seen.pivot(index='user_uid', columns='columns')

In [82]:
result_2['seen_threads']=result_2.apply(lambda row: row.dropna().tolist(), axis=1)

In [83]:
df_test_threads_f['seen_threads'] = result_2[['seen_threads']]

In [84]:
relev = []
for x in range(df_test_threads_f.shape[0]):
    A = list(set(df_test_threads_f['concat_col'].iloc[x]) & (set(df_test_threads_f['sim_threads'].iloc[x]) - set(df_test_threads_f['seen_threads'].iloc[x])))
    relev.append(len(A))

In [85]:
df_test_threads_f['relevant'] = relev
df_test_threads_f['len_concat'] = df_test_threads_f['concat_col'].apply(lambda x: len(x))
df_test_threads_f['len_predict'] = df_test_threads_f['sim_threads'].apply(lambda x: len(x))

In [86]:
df_test_threads_f['precision'] = df_test_threads_f['relevant']/df_test_threads_f['len_predict']
df_test_threads_f['recall'] = df_test_threads_f['relevant']/df_test_threads_f['len_concat']

In [88]:
Precision = df_test_threads_f[['precision']].mean()
Precision

           columns
precision             0.045483
dtype: float64

In [89]:
Recall = df_test_threads_f[['recall']].mean()
Recall

        columns
recall             0.442162
dtype: float64